In [53]:
#import modules
import argparse
import os
import random
import numpy as np
from tkinter.tix import IMAGE

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from glob import glob


# Device
CUDA_DEVICE_NUM = 0
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', DEVICE)
print(torch.cuda.is_available())

# Hyperparameters
LEARNING_RATE = 0.0001
TRAINDATA = "ISIC\ISIC-2017_Training_Data"
TRAINDATA = "ISIC\ISIC-2017_Training_Data"

TESTDATA = "ISIC\ISIC-2017_Test_v2_Data"
VALIDDATA = "ISIC\ISIC-2017_Validation_Data"
NUM_EPOCHS = 5
BATCH_SIZE = 4
WORKERS = 4


Device: cuda:0
True


In [73]:



for file in os.listdir(TRAINDATA+"\ISIC-2017_Training_Data"):
    if file.endswith(".jpg"):
        print(os.path.join(TRAINDATA+"\ISIC-2017_Training_Data", file))
images = sorted(glob(os.path.join(TRAINDATA+"\ISIC-2017_Training_Data", "*.jpg")))
masks = sorted(glob(os.path.join(TRAINDATA, "\ISIC-2017_Training_Data", "*.png")))
print(images)
print(masks)

ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000000.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000001.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000002.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000003.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000004.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000006.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000007.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000008.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000009.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000010.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000011.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000012.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000013.jpg
ISIC\ISIC-2017_Training_Data\ISIC-2017_Training_Data\ISIC_0000014.jpg
ISIC\ISIC-2017_Train

In [13]:

train_dataset = dset.ImageFolder(root=TRAINDATA,
                           transform=transforms.Compose([
                               transforms.ToTensor()
                           ]))
# Create the dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=WORKERS)

test_dataset = dset.ImageFolder(root=TESTDATA,
                           transform=transforms.Compose([
                               transforms.ToTensor()
                           ]))
# Create the dataloader
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=WORKERS)

valid_dataset = dset.ImageFolder(root=VALIDDATA,
                           transform=transforms.Compose([
                               transforms.ToTensor()
                           ]))
# Create the dataloader
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=WORKERS)

#print(train_dataset)
print("Number of train samples: ", len(train_dataset))
#for i in train_dataset:
#   print(i)
counter = 0
for images in train_dataset:  
    counter +=  1
    print(images)
    print('Image batch dimensions:', images.size())
    #print('Image label dimensions:', labels.size())
    #print(labels[:10])
    print(counter)
    if counter > 10:
       break

plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(images[:64], 
                                         padding=2, normalize=True),
                        (1, 2, 0)))




Number of train samples:  4000
(tensor([[[0.9765, 0.9059, 0.9451,  ..., 0.9294, 0.9333, 0.9765],
         [0.8941, 0.6824, 0.6431,  ..., 0.6941, 0.7216, 0.9333],
         [0.9569, 0.6627, 0.5725,  ..., 0.6196, 0.6627, 0.9333],
         ...,
         [0.9412, 0.6980, 0.6627,  ..., 0.7373, 0.7647, 0.9333],
         [0.9098, 0.7137, 0.7137,  ..., 0.7765, 0.7882, 0.9333],
         [0.9843, 0.9294, 0.9765,  ..., 0.9333, 0.9176, 0.9765]],

        [[1.0000, 0.9451, 1.0000,  ..., 1.0000, 0.9804, 1.0000],
         [0.9176, 0.7216, 0.7098,  ..., 0.7961, 0.7843, 0.9725],
         [0.9843, 0.7059, 0.6392,  ..., 0.7412, 0.7451, 0.9961],
         ...,
         [1.0000, 0.7725, 0.7373,  ..., 0.8627, 0.8471, 1.0000],
         [0.9569, 0.7647, 0.7608,  ..., 0.8824, 0.8510, 0.9843],
         [1.0000, 0.9686, 1.0000,  ..., 1.0000, 0.9608, 1.0000]],

        [[1.0000, 0.9843, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9647, 0.7686, 0.7804,  ..., 0.9020, 0.8471, 1.0000],
         [1.0000, 0.7843, 

AttributeError: 'tuple' object has no attribute 'size'

In [6]:
print('Training Set:\n')
for images, labels in train_dataloader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    break
'''
# Checking the dataset
print('\nValidation Set:')
for images, labels in valid_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    #print(labels[:10])
    break

# Checking the dataset
print('\nTesting Set:')
for images, labels in test_loader:  
    print('Image batch dimensions:', images.size())
    print('Image label dimensions:', labels.size())
    #print(labels[:10])
    break
'''
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(images[:64], 
                                         padding=2, normalize=True),
                        (1, 2, 0)))

Training Set:



RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, num_filters):
        super(ConvBlock, self).__init__()
        
        # 2D convolution layer
        self.conv = nn.Conv2d(in_channels, num_filters, kernel_size=3, padding=1)
        
        # Batch normalization
        self.bn = nn.BatchNorm2d(num_filters)
        
        # No need to explicitly define ReLU activation here, as we can use the functional API provided by PyTorch
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        return x
    
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.conv = double_conv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x1 = self.conv(x)
        x2 = self.pool(x1)
        return x1, x2

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_channels):
        super(DecoderBlock, self).__init__()
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.concat = torch.cat
        self.conv = double_conv(out_channels + skip_channels, out_channels) # +skip_channels because of concatenation
        
    def forward(self, x, skip_features):
        x = self.upconv(x)
        x = torch.cat([x, skip_features], dim=1) # concatenate along the channels dimension
        x = self.conv(x)
        return x


In [6]:




class UNet(nn.Module):
    def __init__(self, input_channels):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = double_conv(input_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = double_conv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = double_conv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = double_conv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
        # Bridge
        self.bridge = double_conv(512, 1024)
        # Decoder
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec1 = double_conv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec2 = double_conv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = double_conv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec4 = double_conv(128, 64)
        
        # Final Layer
        self.final = nn.Conv2d(64, 1, kernel_size=1)
        
    def forward(self, x):
        # Encoder
        s1 = self.enc1(x)
        p1 = self.pool1(s1)
        s2 = self.enc2(p1)
        p2 = self.pool2(s2)
        s3 = self.enc3(p2)
        p3 = self.pool3(s3)
        s4 = self.enc4(p3)
        p4 = self.pool4(s4)
        
        # Bridge
        b1 = self.bridge(p4)
        
        # Decoder
        d1 = self.up1(b1)
        d1 = torch.cat((s4, d1), dim=1)
        d1 = self.dec1(d1)
        d2 = self.up2(d1)
        d2 = torch.cat((s3, d2), dim=1)
        d2 = self.dec2(d2)
        d3 = self.up3(d2)
        d3 = torch.cat((s2, d3), dim=1)
        d3 = self.dec3(d3)
        d4 = self.up4(d3)
        d4 = torch.cat((s1, d4), dim=1)
        d4 = self.dec4(d4)
        out = self.final(d4)
        
        return torch.sigmoid(out)


model = UNet(input_channels=3) 
model = model.to(DEVICE)

TRAIN 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, 
                            momentum = 0.9, weight_decay = 5e-4)

total_steps = len(train_dataloader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr = LEARNING_RATE, 
                                steps_per_epoch = total_steps, epochs = NUM_EPOCHS)

In [7]:
model.train()
print("> Training")


> Training


In [9]:

for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        #print(images.shape)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
                    print (f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i+1}/{total_steps}], Loss: {loss.item():.5f}')
        scheduler.step()

print("Training took " +  " secs")
print("> Testing")

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 265, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 142, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples